# Homework 12

https://scikit-learn.org/0.15/modules/scaling_strategies.html#incremental-learning

* Implement a mini batch functionality to train a regressor.
    - (Optional) If anyone want to do this in a pipeline can do this: https://koaning.github.io/tokenwiser/api/pipeline.html

* Save model, load the model again and test it on `X_test` __Do NOT commit the pickle file__

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
def test_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)

    df = df.sample(5000, random_state=100).reset_index(drop=True)
    
    y = df['sellingprice']
    df.drop('sellingprice', axis=1, inplace=True)
    X = df
    
    return X,y

def partial_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)
   
    while(True):
        yield df.sample(100).reset_index(drop=True)
        
gen = partial_df()

In [3]:
X_test, y_test = test_df()

In [4]:
# each time you call this you will get a new slice of the dataframe.
next(gen)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2014,Kia,Forte,LX,Sedan,automatic,knafx4a64e5217171,ca,4.4,11787.0,brown,black,"kia motors america, inc",12450,12700,Tue Jan 20 2015 04:30:00 GMT-0800 (PST)
1,2013,MINI,Cooper,S,Hatchback,manual,wmwsv3c56dt478556,fl,4.7,8191.0,gray,black,subaru of pembroke pines,18400,17000,Fri Jan 16 2015 01:30:00 GMT-0800 (PST)
2,2013,Ford,Escape,SEL,SUV,automatic,1fmcu0h91dub17486,tx,3.9,87706.0,black,tan,ge fleet services for itself/servicer,13500,12600,Thu Jan 08 2015 11:00:00 GMT-0800 (PST)
3,2012,BMW,5 Series,535i xDrive,Sedan,automatic,wbafu7c55cdu58819,nv,2.8,30562.0,gray,beige,financial services remarketing (lease),31200,28250,Fri Jun 05 2015 05:00:00 GMT-0700 (PDT)
4,2004,Chevrolet,Tahoe,LS,SUV,automatic,1gnec13t24r283888,wa,2,118224.0,green,tan,primeritus remarketing/salal credit union,6425,4200,Wed May 27 2015 06:20:00 GMT-0700 (PDT)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2010,GMC,Yukon XL,SLT 1500,suv,automatic,1gkukke34ar248019,tn,3.6,76205.0,burgundy,black,wholesale inc # 2,22900,23750,Wed May 27 2015 03:30:00 GMT-0700 (PDT)
96,2010,Nissan,Maxima,3.5 SV,Sedan,automatic,1n4aa5ap7ac845671,wi,4.4,98079.0,gray,black,bmw alphera/alphera financial services,11050,10000,Wed Jan 28 2015 02:00:00 GMT-0800 (PST)
97,2004,Nissan,Xterra,XE,SUV,automatic,5n1ed28t54c615406,ga,3.6,151531.0,red,gray,o'brien's automotive group,2875,4100,Tue Feb 03 2015 05:10:00 GMT-0800 (PST)
98,2005,chrysler,pacifica,fwd v6,NaN,automatic,2c4gm48l45r453396,ca,NaN,127019.0,white,tan,lithia nissan of fresno,2250,2000,Thu Dec 18 2014 13:00:00 GMT-0800 (PST)


In [5]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [6]:
# Defining the missing value imputation strategies

num_features = [ 'odometer', 'year']
cat_features = ['make', 'model', 'trim', 'body', 'transmission', 'vin', 'state', 'condition', 'color', 'interior', 'seller', 'saledate']
num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

# Defining the transformers
num_transformer = Pipeline(steps=[
    ('imputer', num_imputer),
    ('scaler', StandardScaler())])

cat_transformer = Pipeline(steps=[
    ('imputer', cat_imputer),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Defining the preprocessor
prepro = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)])

# Fitting the preprocessor on the training data
prepro.fit(X_test)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['odometer', 'year']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['make', 'model', 'trim', 'body',
                                  'transmission', 'vin', 'state', 'condition',
                                  'color', 'interior', 'seller', 'saledate'])])

The missing value imputation algorithms are first specified independently for the categorical and numerical characteristics. The names of the numerical and categorical characteristics are listed in the num_features and cat_features lists, respectively. The missing value imputation approach for the numerical and categorical features is specified by the num_imputer and cat_imputer objects, respectively. Here, categorical features are handled by the most_frequent method while numerical features are handled by the median strategy.

The num_transformer and cat_transformer pipelines are the next two to be defined. The num_transformer pipeline adjusts numerical features using the StandardScaler() function after using the num_imputer approach to fill in missing values. Similar to this, the cat_transformer pipeline uses the cat_imputer technique to fill in missing values in categorical features before utilizing the OneHotEncoder() method to apply one-hot encoding.

The ColumnTransformer object prepro, which employs the num_transformer pipeline to transform numerical data and the cat_transformer pipeline to change categorical features, is defined last. The entire dataset may be transformed using the prepro object, which automatically applies the proper preprocessing processes to the different feature types.

In [7]:
Regressor1 = SGDRegressor(loss='squared_error', random_state=100)

In [8]:
# Mini-batch training loop
batch_size = 50
total_batches = 100
n_epoch = 100
for epoch in range(n_epoch):
    for _ in range(total_batches):
        batch_df = next(gen)
        X_b = batch_df.drop('sellingprice', axis=1)
        Y_b = batch_df['sellingprice']

        # Applying  preprocessor to the mini-batch
        X_b_pro = prepro.transform(X_b)

        # Handling the missing values in the mini-batch
        X_b_pro = np.nan_to_num(X_b_pro)

        Regressor1.partial_fit(X_b_pro, Y_b)

The loop iterates over a predetermined total number of mini-batches (total_batches) and runs for a predetermined number of epochs (n_epoch). To get the subsequent mini-batch of data, the generator gen's next() function is called after each iteration. The sellingprice column is assigned to Y_b, and the features are obtained using the drop() function of the batch_df object.

The transform() function is used by the prepro object to preprocess the numerical and categorical characteristics in the mini-batch. Any NaN or infinity values in the preprocessed features are changed to 0 using the nan_to_num() function.

Finally, the target variable Y_b and the preprocessed features X_b_pro are sent to the partial_fit() function of the Regressor1 object. The partial_fit() function may be used repeatedly for each mini-batch of data to incrementally update the model weights depending on the mini-batch of data.

In [9]:
import joblib
joblib.dump(Regressor1, 'regressor_model.joblib')

# Loading the model
l_Regressor1 = joblib.load('regressor_model.joblib')

# Preprocessing the test data
X_test_pro = prepro.transform(X_test)

# Testing the model on X_test
y_pred1 = l_Regressor1.predict(X_test_pro)

# Print the predicted values
print(y_pred1)
l_Regressor1.score(X_test_pro,y_test)

[14437.42308783 20219.86439306 15245.80875425 ... 19778.14678419
 19242.62675932 16627.56455931]


0.8108290402263937

The code above saves the Regressor1 model using joblib, loads it into l_Regressor1, preprocesses the test data X_test using prepro, makes predictions using l_Regressor1 on the preprocessed test data X_test_pro, and outputs the projected values.

The score of the loaded model l_Regressor1 on the preprocessed test data X_test_pro is calculated and displayed in the last line.